# 들어가며
이 내용은 [Django ORM Cookbook](https://django-orm-cookbook-ko.readthedocs.io/en/latest/)을 참고하여 작성하였습니다.

## 1. 장고 ORM이 실행하는 실제 SQL 질의문을 확인 하기
`queryset.query`의 `str`을 확인하면 된다.

In [4]:
queryset = User.objects.all()
str(queryset.query)

'SELECT "users"."id", "users"."email", "users"."name" FROM "users"'

## 2. OR, AND, NOT 연산 사용하기

### 1. OR 연산
- queryset_1 | queryset_2
- filter(Q(조건1) | Q(조건2))

In [6]:
queryset = User.objects.filter(id=1) | User.objects.filter(id=2)
str(queryset.query)

'SELECT "users"."id", "users"."email", "users"."name" FROM "users" WHERE ("users"."id" = 1 OR "users"."id" = 2)'

In [8]:
queryset = User.objects.filter(Q(id=1) | Q(id=2))
str(queryset.query)

'SELECT "users"."id", "users"."email", "users"."name" FROM "users" WHERE ("users"."id" = 1 OR "users"."id" = 2)'

### 2. AND 연산
- queryset_1 & queryset_2
- filter(Q(조건1) & Q(조건2))
- filter(조건1, 조건2)

In [14]:
queryset = User.objects.filter(id__gte=1) & User.objects.filter(id__lte=2)
str(queryset.query)

'SELECT "users"."id", "users"."email", "users"."name" FROM "users" WHERE ("users"."id" >= 1 AND "users"."id" <= 2)'

In [15]:
queryset = User.objects.filter(Q(id__gte=1) & Q(id__lte=2))
str(queryset.query)

'SELECT "users"."id", "users"."email", "users"."name" FROM "users" WHERE ("users"."id" >= 1 AND "users"."id" <= 2)'

In [16]:
queryset = User.objects.filter(id__gte=1, id__lte=2)
str(queryset.query)

'SELECT "users"."id", "users"."email", "users"."name" FROM "users" WHERE ("users"."id" >= 1 AND "users"."id" <= 2)'

### 3.NOT 연산
- exclude(조건)
- filter(~Q(조건))

In [17]:
queryset = User.objects.exclude(id=1)
str(queryset.query)

'SELECT "users"."id", "users"."email", "users"."name" FROM "users" WHERE NOT ("users"."id" = 1)'

In [18]:
queryset = User.objects.filter(~Q(id=1))
str(queryset.query)

'SELECT "users"."id", "users"."email", "users"."name" FROM "users" WHERE NOT ("users"."id" = 1)'

## 3. 필요한 열만 골라 조회하기
- 쿼리셋의 values 메서드와 values_list 메서드 - dictionary나 tuple형태로 가져옴
- only 메서드 - 객체로 가져오는데, id도 같이 가져옴

In [32]:
queryset = User.objects.filter(id=1).values('name')
print(queryset.query, queryset)

SELECT "users"."name" FROM "users" WHERE "users"."id" = 1 <QuerySet [{'name': '유져0'}]> 유져0


In [26]:
queryset = User.objects.filter(id=1).values_list('name')
print(queryset.query, queryset)

SELECT "users"."name" FROM "users" WHERE "users"."id" = 1 <QuerySet [('유져0',)]>


In [43]:
queryset = User.objects.filter(id=1).only('name')
print(queryset.query, queryset[0])

SELECT "users"."id", "users"."name" FROM "users" WHERE "users"."id" = 1 User object (1)


##### only 사용시 주의점
지정하지 않은 필드를 접근 할 경우 내부적으로 해당 필드 추가해서 SELECT 문이 한번 더 실행
```python
qs = User.objects.filter(id=1).only("name")
qs[0].email
# SELECT "users"."id", "users"."name" FROM "users" WHERE "users"."id" = 1 LIMIT 1;
# SELECT "users"."id", "users"."email" FROM "users" WHERE "users"."id" = 1 LIMIT 21;
```

## 4. 서브쿼리 사용하기
- [서브쿼리 - 제타위키](https://zetawiki.com/wiki/서브쿼리)
- [subquery-expressions](https://docs.djangoproject.com/en/4.0/ref/models/expressions/#subquery-expressions)
- `Subquery` 객체를 사용해서 구현한다

In [46]:
queryset = User.objects.filter(id__in=Subquery(User.objects.filter(id__gt=10).values('id')))
str(queryset.query)

'SELECT "users"."id", "users"."email", "users"."name" FROM "users" WHERE "users"."id" IN (SELECT U0."id" FROM "users" U0 WHERE U0."id" > 10)'

Outer queryset을 참고하고 싶으면 [OuterRef](https://docs.djangoproject.com/en/4.0/ref/models/expressions/#django.db.models.OuterRef)를 사용한다.

아래 예시에서 OuterRef("pk")가 의미하는 것은 Seller의 pk이다.

In [87]:
product = Product.objects.filter(related_seller=OuterRef("pk")).order_by('-price')
seller = Seller.objects.all().annotate(most_expensive_product_name=Subquery(product.values('name')[:1]))

print(seller.query)
print(seller.values()[:1])

SELECT "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number", (SELECT U0."name" FROM "products" U0 WHERE U0."related_seller_id" = "sellers"."id" ORDER BY U0."price" DESC LIMIT 1) AS "most_expensive_product_name" FROM "sellers"
<QuerySet [{'id': 72, 'email': 'seller72@django.com', 'name': '판매자1', 'phone_number': '010-1234-0072', 'most_expensive_product_name': '상품37'}]>


## 5.필드의 값을 서로 비교하는 방법
- [F객체](https://docs.djangoproject.com/en/4.0/ref/models/expressions/#f-expressions)를 사용한다.
- 모델 필드 값을 참조하고 실제로 데이터베이스에서 파이썬 메모리로 가져올 필요 없이 이 필드 값을 사용하여 데이터베이스 작업을 수행할 수 있게 한다.
- Django는 F() 개체를 사용하여 데이터베이스 수준에서 필요한 작업을 설명하는 SQL 식을 생성한다.

In [90]:
queryset = User.objects.filter(email=F('name')).only('id')
str(queryset.query)

'SELECT "users"."id" FROM "users" WHERE "users"."email" = "users"."name"'

### F객체의 장점
- 불필요한 쿼리를 줄일 수 있다.

만약 상품의 가격을 1000증가 시키는 로직이 필요하다고 가정하자

- F 객체를 사용하지 않으면 select문, update문 각 1번씩 총 2번의 쿼리를 수행해야된다.
- 반면에 F객체를 사용하면 update문 한번의 쿼리를 수행하면 된다.

#### F객체 미사용 
- F 객체를 미 사용시에는 Update문에서 price를 1300으로 갱신하는 즉 `값으로` 갱신하는 것을 확인 할 수 있다.
- 즉 select문으로 값을 알아낸다 -> 알아낸 값에 1000을 더해서 값을 업데이트 한다
```python
In [28]: qs1 = Product.objects.filter(id__lte=5)
In [29]: for product in qs1:
    ...:     product.price += 1000
    ...:     product.save()
```
```sql
SELECT "products"."id", "products"."name", "products"."price", "products"."related_seller_id" FROM "products" LIMIT 5;
UPDATE "products" SET "name" = '상품1', "price" = 1300, "related_seller_id" = 1 WHERE "products"."id" = 1;
UPDATE "products" SET "name" = '상품2', "price" = 1300, "related_seller_id" = 2 WHERE "products"."id" = 2;
UPDATE "products" SET "name" = '상품3', "price" = 1300, "related_seller_id" = 3 WHERE "products"."id" = 3;
UPDATE "products" SET "name" = '상품4', "price" = 1300, "related_seller_id" = 4 WHERE "products"."id" = 4;
UPDATE "products" SET "name" = '상품5', "price" = 1300, "related_seller_id" = 5 WHERE "products"."id" = 5;
```

#### F객체 사용
- 반면 F객체를 사용하면 현재 필드 값을 참조하여 그 값에 1000을 더하는 로직인 것을 확인 할 수 있다.
```python
qs1 = Product.objects.filter(id__lte=5).update(price=F("price") + 1000)
```
```sql
UPDATE "products" SET "price" = ("products"."price" + 1000) WHERE "products"."id" <= 5;
```


## 6. 두 모델을 Join하는 방법

### 1.selected_related 사용

In [107]:
sq = Product.objects.select_related("related_seller")
str(sq.query)


'SELECT "products"."id", "products"."name", "products"."price", "products"."related_seller_id", "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number" FROM "products" INNER JOIN "sellers" ON ("products"."related_seller_id" = "sellers"."id")'

2. 더블스코어로(`__`) 연결된 테이블의 필드 참조

In [104]:
sq = Product.objects.filter(name='상품1, related_seller__name='판매자1')
str(sq.query)

'SELECT "products"."id", "products"."name", "products"."price", "products"."related_seller_id" FROM "products" INNER JOIN "sellers" ON ("products"."related_seller_id" = "sellers"."id") WHERE ("products"."name" = 상품1 AND "sellers"."name" = 판매자1)'

## 7. N번째의 데이터를 가져오는 방법(Limit, Offset)
- 쿼리셋에 인덱스 연산`[]`을 지정한다.
- 이때 전체 데이터를 가져온 후, Django에서 인덱싱 하는 것이 아니라 SQL 단계에서 Limit, Offset을 적용해서 실제로 필요한 데이터만 가져온다
- python에 인덱싱 처럼 사용하면 되고 Django에서 알아서 해당 번째 데이터를 가져올 수 있도록 Limit, Offset을 조정한다.
- 다만 python의 인덱싱과 다른점은
  - 데이터가 없는 범위를 지정해도 에러가 나지 않는다 - 단 한개로 지정 `ex: User.objects.all()[[50]` 했을때 51번째 데이터가 없다면 에러가 난다.
  - 리버스 인덱스(-1, -2)를 사용할 수 없다. - 아마도 Django입장에서는 DB를 접근하지 않은 상태에서 해당 쿼리 실행결과의 총 갯수를 알 수 없으므로 Limit, Offset으로 치환 할 수 없어서로 추측된다.

In [126]:
# 범위를 한개로 지정했을 때는 에러가 날 수 있다. - SELECT문은 정상 실행하였고, Django에서 데이터를 뽑아줄 때 에러가 남
# 범위가 한개면 반환값이 queryset이 아닌 model 객체이다.
qs = User.objects.all()[1000]

IndexError: list index out of range

In [127]:
# 범위를 한개로 지정 안하면 단순히 결과가 없는 상태다.
qs = User.objects.all()[1000:2000]
print(qs.query, qs)

SELECT "users"."id", "users"."email", "users"."name" FROM "users" LIMIT 1000 OFFSET 1000 <QuerySet []>


## 8. 집계함수 사용하기
- [aggregate](https://docs.djangoproject.com/en/4.0/topics/db/aggregation/)를 사용한다.

In [139]:
# 총 합
Product.objects.all().aggregate(total=Sum('price'))

{'total': 26900}

```sql
SELECT AVG("products"."price") AS "total" FROM "products
```

In [140]:
# 최대 값
Product.objects.all().aggregate(most_expensive=Max('price'))

{'most_expensive': 2300}

```sql
SELECT MAX("products"."price") AS "most_expensive" FROM "products";
```

## 9. 정렬하기
- `order_by`메서드를 사용해서 정렬할 수 있다.
- 정렬하고 싶은 필드명을 입력하는데, 필드명 앞에 `-`를 붙이면 내림차순이 된다.

In [144]:
User.objects.all().order_by('id')[:10] # id기준으로 오름차순 정렬

<QuerySet [<User: User object (371)>, <User: User object (372)>, <User: User object (373)>, <User: User object (374)>, <User: User object (375)>, <User: User object (376)>, <User: User object (377)>, <User: User object (378)>, <User: User object (379)>, <User: User object (380)>]>

In [143]:
User.objects.all().order_by('-id')[:10] # id기준으로 내림차순 정렬

<QuerySet [<User: User object (400)>, <User: User object (399)>, <User: User object (398)>, <User: User object (397)>, <User: User object (396)>, <User: User object (395)>, <User: User object (394)>, <User: User object (393)>, <User: User object (392)>, <User: User object (391)>]>